In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [18]:
df_features = pd.read_csv('../data/features.csv')
df_sales = pd.read_csv('../data/sales.csv')
df_stores = pd.read_csv('../data/stores.csv')

In [19]:
df_features['Date'] = pd.to_datetime(df_features['Date'], format='%d/%m/%Y')
df_sales['Date'] = pd.to_datetime(df_sales['Date'], format='%d/%m/%Y')


In [20]:
df_features = df_features.rename(columns={'IsHoliday': 'IsHolidayFeatures'})
df_combined = pd.merge(df_sales, df_stores, on='Store', how='left')
df_final = pd.merge(df_combined, df_features, on=['Store', 'Date'], how='left')
df_final['week'] = df_final['Date'].dt.isocalendar().week
df_final['month'] = df_final['Date'].dt.month
df_final['day'] = df_final['Date'].dt.day
df_final['original_date'] = df_final['Date']
df_final.set_index('Date', inplace=True)
df_final = df_final.sort_index()


In [21]:
df_final.drop(columns=['IsHolidayFeatures'], axis=1, inplace=True)

In [22]:
shiftable_columns = ['Weekly_Sales', 'Temperature', 'MarkDown1','MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI','Unemployment']

In [23]:
for col in shiftable_columns:
    df_final[f'lag_{col}'] = df_final.groupby(['Store', 'Dept'])[f'{col}'].shift(52)
df_shifted = df_final.drop(shiftable_columns, axis = 1)
df_shifted =df_shifted.dropna(subset=['lag_Weekly_Sales'])
df_shifted.rename(columns={col: col[4:] for col in df_shifted.columns if col.startswith('lag_')}, inplace=True)


In [24]:
df_shifted.to_csv('improved_data.csv', index=False)
